# Build a Bag-of-Words (BoW) Sentiment Classifier

This notebook is using the same data and supporting code as the rule-based classifier.
Please see that notebook for details on the data.

Unlike the rule-based classifier, this BoW classifier will use automatically extracted features and _learn_ the weights on these features.
Our features will be specifically be count vectors where each index refers to the number of a given word found in the input string.
This is referred to of as a "bag of words" because it's like throwing all of the words into a bag and counting them -- while this method is simple, the main disadvantage is that we lose all structural information present in the sentence(s).
We can then use our training data to learn weights between the individual words and our positive, negative, and neutral labels.

## Data Reading

Read in the data from the training and dev (or finally test) sets

In [ ]:
def read_XY_data(filename):
    X_data = []
    Y_data = []
    with open(filename, 'r') as f:
        for line in f:
            label, text = line.strip().split(' ||| ')
            X_data.append(text)
            Y_data.append(int(label))
    return X_data, Y_data

In [ ]:
X_train, Y_train = read_XY_data('../data/sst-sentiment-text-threeclass/train.txt')
X_test, Y_test = read_XY_data('../data/sst-sentiment-text-threeclass/dev.txt')

In [ ]:
def tokenize(datum):
    # Split string into words
    return datum.split(" ")

def build_feature_map(X):
    # We need to assign an index to each word in order to build the count vector.
    # We start by gathering a set of all word types in the training data.
    word_types = set()
    for datum in X:
        for word in tokenize(datum):
            word_types.add(word)
    # Create a dictionary keyed by word mapping it to an index
    return {word: idx for idx, word in enumerate(word_types)}
            

from scipy.sparse import dok_matrix

def extract_features(word_to_idx, X):
    # We are using a sparse matrix from scipy to avoid creating an 8000 x 18000 matrix
    features = dok_matrix((len(X), len(word_to_idx)))
    for i in range(len(X)):
        for word in tokenize(X[i]):
            if word in word_to_idx:
                # Increment the word count if it is present in the map.
                # Unknown words are discarded because we would not have
                # a learned weight for them anyway.
                features[i, word_to_idx[word]] += 1
    return features


In [ ]:
sample_data = [
    "When is the homework due ?",
    "When are the TAs' office hours ?",
    "How hard is the homework ?",
]

word_to_idx = build_feature_map(sample_data)
print(word_to_idx)
print()

features = extract_features(word_to_idx, sample_data)
print(features)

Now let's run the feature extractor on the actual data.

In [ ]:
# Build the map based on the training data
word_to_idx = build_feature_map(X_train)

print(f"Unique word types in X_train: {len(word_to_idx)}")
print("Sample words:")
print(list(word_to_idx.keys())[:20])

In [ ]:
# Convert our strings into count vectors
X_train_vec = extract_features(word_to_idx, X_train)
X_test_vec = extract_features(word_to_idx, X_test)

from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(tol=1e1)
classifier.fit(X_train_vec, Y_train)

# Create a truncated version of the training set so we have a second model to compare to
X_train_vec_trunc = extract_features(word_to_idx, [x[:100] for x in  X_train])
classifier_trunc = LogisticRegression(tol=1e1)
classifier_trunc.fit(X_train_vec_trunc, Y_train)

## Testing for statistical significance

Now that we have trained our two models, it is tempting to just evaluate the accuarcy and see which one is better.
The problem here is that we do not know if the difference in accuracy between the two models is statistically significant or not -- the difference might just be random chance.

There are many different kinds of tests for statistical significance, but we will implement and perform just one here: pairwise bootstrapping.
Bootstraping, generally speaking, refers to randomly sampling a subset of some data, calculating a staistic on the subset (e.g., taking their mean) and looking at the distribution of that statistic.
With pairwise bootstrapping, we are going to randomly select a subset of test labels for the true labels and the two system we want to compare; we'll then see which system has the higher accuracy.
We'll do this a number of times (i.e., 10,000) to get a better sense of the statistical significance of the difference between our systems.

In [ ]:
import numpy as np

rng = np.random.default_rng()


def eval_with_paired_bootstrap(gold, sys1, sys2, num_samples=10000, sample_ratio=0.5):
    """Evaluate with paired boostrap
    This compares two systems, performing a significance tests with
    paired bootstrap resampling to compare the accuracy of the two systems.

    Parameters
    ----------
    gold
      The correct labels
    sys1
      The output of system 1
    sys2
      The output of system 2
    num_samples
      The number of bootstrap samples to take
    sample_ratio
      The ratio of samples to take every time

    """
    assert len(gold) == len(sys1)
    assert len(gold) == len(sys2)

    gold = np.array(gold)
    sys1 = np.array(sys1)
    sys2 = np.array(sys2)

    sys1_scores = []
    sys2_scores = []
    wins = [0, 0, 0]
    n = len(gold)

    for _ in range(num_samples):
        # Subsample the gold and system outputs
        subset_idxs = rng.choice(n, int(n * sample_ratio), replace=True)
        sys1_score = (sys1[subset_idxs] == gold[subset_idxs]).mean()
        sys2_score = (sys2[subset_idxs] == gold[subset_idxs]).mean()

        if sys1_score > sys2_score:
            wins[0] += 1
        elif sys1_score < sys2_score:
            wins[1] += 1
        else:
            wins[2] += 1

        sys1_scores.append(sys1_score)
        sys2_scores.append(sys2_score)

    # Print win stats
    wins = [x / float(num_samples) for x in wins]
    print("Win ratio: sys1=%.3f, sys2=%.3f, tie=%.3f" % (wins[0], wins[1], wins[2]))
    if wins[0] > wins[1]:
        print("(sys1 is superior with p value p=%.3f)\n" % (1 - wins[0]))
    elif wins[1] > wins[0]:
        print("(sys2 is superior with p value p=%.3f)\n" % (1 - wins[1]))

    # Print system stats
    sys1_scores.sort()
    sys2_scores.sort()
    print(
        "sys1 mean=%.3f, median=%.3f, 95%% confidence interval=[%.3f, %.3f]"
        % (
            np.mean(sys1_scores),
            np.median(sys1_scores),
            sys1_scores[int(num_samples * 0.025)],
            sys1_scores[int(num_samples * 0.975)],
        )
    )
    print(
        "sys2 mean=%.3f, median=%.3f, 95%% confidence interval=[%.3f, %.3f]"
        % (
            np.mean(sys2_scores),
            np.median(sys2_scores),
            sys2_scores[int(num_samples * 0.025)],
            sys2_scores[int(num_samples * 0.975)],
        )
    )


In [28]:
cls_preds = classifier.predict(X_test_vec)
cls_trunc_preds = classifier_trunc.predict(X_test_vec)
baseline_preds = np.ones_like(cls_preds)

eval_with_paired_bootstrap(Y_test, cls_preds, baseline_preds)
print()
eval_with_paired_bootstrap(Y_test, cls_preds, cls_trunc_preds)

Win ratio: sys1=1.000, sys2=0.000, tie=0.000
(sys1 is superior with p value p=0.000)

sys1 mean=0.597, median=0.596, 95% confidence interval=[0.569, 0.625]
sys2 mean=0.403, median=0.404, 95% confidence interval=[0.375, 0.433]

Win ratio: sys1=0.777, sys2=0.169, tie=0.053
(sys1 is superior with p value p=0.223)

sys1 mean=0.596, median=0.596, 95% confidence interval=[0.567, 0.627]
sys2 mean=0.588, median=0.589, 95% confidence interval=[0.558, 0.618]


While our full classifier is significantly better than the baseline, we cannot say that the full model is significantly better than the truncated model since the p-value is above 0.05 (the lower, the more significant).